## 1: Retrieve Episodes

In [24]:
import pandas as pd

In [25]:
narcos_url = "https://transcripts.foreverdreaming.org/viewforum.php?f=1394"

In [72]:
from bs4 import BeautifulSoup
import requests

# send a GET request to the website
url = narcos_url
response = requests.get(url)

# create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# find the element(s) you want to extract
title_element = soup.find('title')

# extract the text content from the element
title_text = title_element.text


In [73]:
title_text

'Narcos - Transcripts - Forever Dreaming'

In [74]:
title_element

<title>Narcos - Transcripts - Forever Dreaming</title>

In [86]:
links = soup.find_all('a', class_ = "topictitle")
urls = []
for link in links:
    l = "https://transcripts.foreverdreaming.org/" + link["href"][2:]
    urls.append(l)
df_urls = pd.DataFrame({"urls": urls})

In [92]:
## df_urls

In [90]:
episode_links = df_urls
# save as pickle
episode_links.to_pickle("links_to_episodes.pkl")
# read from pickle
episode_links = pd.read_pickle("links_to_episodes.pkl")

## 2: EXTRACT TRANSCRIPTS BY EPISODE


In [165]:
len(episode_links)

31

In [166]:
# The first URL is not an episode so we iterate from the second one 
ep_transcripts = []
for index, row in episode_links[1:].iterrows():
    # print(row["urls"])
    link_episode = row["urls"]
    resp = requests.get(link_episode)
    # create a BeautifulSoup object
    soup_episode = BeautifulSoup(resp.text, 'html.parser')
    title_episode = soup_episode.find('h2').text
    transcript_episode = soup_episode.find_all('div', class_ = "content")
    ep_transcripts.append({"title": title_episode, "transcript": transcript_episode})

In [159]:
## Convert to data frame
df_ep_transcripts = pd.DataFrame(ep_transcripts)

In [167]:
df_ep_transcripts

,title,transcript
0,03x10 - Going Back to Cali,"[[Hey!, [], \n, [], \nHey, ladies., [], \n, []..."
1,03x09 - Todos Los Hombres del Presidente,"[[[IN SPANISH] Sir?, [], \n, [], \nThe telepho..."
2,03x08 - Convivir,"[[[grunts], [], \n, [], \n[in English] Come. I..."
3,03x07 - Sin Salida,"[[[church congregation singing in Spanish], []..."
4,03x06 - Best Laid Plans,"[[[PEÑA IN ENGLISH] The Cali cartel, [], \nis ..."
5,03x05 - MRO,"[[[water dripping], [], \n, [], \n[buzzer soun..."
6,03x04 - Checkmate,"[[[engine turns over], [], \n, [], \n[Peña in ..."
7,03x03 - Follow the Money,"[[Amado Carrillo Fuentes!, [], \n, [], \nThe L..."
8,03x02 - The Cali KGB,"[[Careful. You already lost one., [], \n, [], ..."
9,03x01 - The Kingpin Strategy,"[[Thank you for being here with us,, [], \n, [..."


## 3: Clean up the data

In [182]:
#Identify all the regular expression such as  <*> and substitute with empty space. 
import re
def cleanhtml(raw_html): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', str(raw_html))
    return cleantext

In [183]:
df_ep_transcripts['clean_transcript'] = df_ep_transcripts.apply(lambda row : cleanhtml(row["transcript"][0]), axis = 1)

['Hey!\n\nHey, ladies.\n\nAre you looking for something?\n\nWe\'re just taking a ride.\n\nCan\'t we pass?\n\nNo, sweetheart. You have to go back.\n\nListen...\n\nDo you want to hang out later\nback in town?\n\nWhat?\nI said...\n\nNo, no, no!\n\nJulio.\nPacho, you f*ck.\n\nf*ck dog!\n\nLet me go!\n\nLet go of me!\n\nLet me go!\n\nLet go...!\n\nYou.\n\nMrs. Salazar.\n\nDo you know what makes me happy, Pacho?\n\nMakes me very happy?\n\nThat my sons didn\'t k*ll\nyour brother Alvaro.\n\nBecause he\'s going to live\nthe rest of his life...\n\nas half a man. Half man, half...\n\nHelp!\n\nHelp!\n\nthe great event of the surrender\n\nof Helmer Herrera, to Colombian justice.\n\nAnd this incident\nmarked the end of the Cali cartel...\n\nwho were responsible,\naccording to international sources,\n\nfor more than 80%\nof the cocaine tr*ffick in the world.\n\nThroughout my campaign and presidency,\n\nI committed to dismantling\nall narcotrafficking networks,\n\nand we\'re doing it.\n\nThe Colombian

## 4: Save to CSV

In [186]:
# save the DataFrame to a CSV file
df_ep_transcripts.to_csv('narcos_episode_data.csv', index=False)